In [5]:
# !wget -O train.zip --no-check-certificate https://huggingface.co/datasets/ccdv/arxiv-summarization/resolve/main/train.zip?download=true
# !Expand-Archive -Path "train.zip"
# !move train.txt train.json
# !del train.zip
# import requests
# from zipfile import ZipFile
# import os

# # Download the file
# url = "https://huggingface.co/datasets/ccdv/arxiv-summarization/resolve/main/train.zip?download=true"
# response = requests.get(url)
# with open("train.zip", "wb") as file:
#     file.write(response.content)

# # Unzip the file
# with ZipFile('train.zip', 'r') as zip_ref:
#     zip_ref.extractall('.')

# # Move and rename the file (adjust the filenames if necessary)
# os.rename('train.txt', 'train.json')  # Make sure the source filename is correct

# # Delete the zip file
# os.remove('train.zip')

# !powershell -Command "Invoke-WebRequest -Uri 'https://huggingface.co/datasets/ccdv/arxiv-summarization/resolve/main/train.zip?download=true' -OutFile 'train.zip'"
# !powershell -Command "Expand-Archive -Path 'train.zip' -DestinationPath '.'"
# !move train.txt train.json
# !del train.zip


In [6]:
# # load papers in json format from each line of the extracted file
# import json
# papers = []
# article_ids = []
# f = 1
# with open('test.txt', 'r') as file:
#     for line in file:
#         try:
#             # Parse the JSON data from each line
#             paper_data = json.loads(line)
#             if f == 1:
#               print("paper_data: ", paper_data)
#               f += 1
#             # Extract the arXiv ID and append to the list
#             if 'article_id' in paper_data:
#                 article_ids.append(paper_data['article_id'])
#             papers.append(paper_data)
#         except json.JSONDecodeError as e:
#             # Output an error message if a line is not valid JSON
#             print(f"Error parsing JSON for line: {line}")
#             print(str(e))


In [7]:
# # Print stuff
# json_formatted_str = json.dumps(papers[1]["sections"][1], indent=2)
# print(json_formatted_str)

## LSA Summarization

In [8]:
# Function to remove stop-words
import nltk
import numpy as np
import re

nltk.download('stopwords')
nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # print("tokens: ", tokens)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # print("filtered tokens: ", filtered_tokens)
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    #print("doc after joining: ", doc)
    return doc
normalize_corpus = np.vectorize(normalize_document)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sai\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
from scipy.sparse.linalg import svds

def low_rank_svd(matrix, singular_count=2):
    u, s, vt = svds(matrix, k=singular_count)
    return u, s, vt

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
# Get top n sentences. Default value of n is 4.
def get_sentences_top(sentences, num_sentences=4):
    norm_sentences = normalize_corpus(sentences)
    norm_sentences = list(filter(None, norm_sentences))
    #print("norm sentences: ", norm_sentences)
    tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
    if len(norm_sentences) == 0:
        return np.array([])
    dt_matrix = tv.fit_transform(norm_sentences)
    #print("dt matrix: ", dt_matrix)
    dt_matrix = dt_matrix.toarray()
    #print("dt matrix after converting it to array: ", dt_matrix)
    vocab = tv.get_feature_names_out()
    # print("Vocab: ", vocab)
    td_matrix = dt_matrix.T
    #print(td_matrix.shape)
    pd.DataFrame(np.round(td_matrix, 2), index=vocab).head(10)
    num_sentences = min(num_sentences, len(sentences))
    #print("num_sentences", num_sentences)
    num_topics = 3
    if(len(norm_sentences) <= num_topics):
        return np.array(sentences)

    u, s, vt = low_rank_svd(td_matrix, singular_count=num_topics)
    #print(u.shape, s.shape, vt.shape)
    term_topic_mat, singular_values, topic_document_mat = u, s, vt

    sv_threshold = 0.5
    min_sigma_value = max(singular_values) * sv_threshold
    singular_values[singular_values < min_sigma_value] = 0
    salience_scores = np.sqrt(np.dot(np.square(singular_values),
                                 np.square(topic_document_mat)))
    top_sentence_indices = (-salience_scores).argsort()[:num_sentences]
    top_sentence_indices.sort()
    return np.array(sentences)[top_sentence_indices]

In [11]:
# Define a method to perform extractive summarization)
def extractive(document_sections, num_sentences=4):
    overall_summary = ""
    for section in document_sections:
        summary = get_sentences_top(section, num_sentences)
        overall_summary += " ".join(summary) + " "
    return overall_summary

# combine sentences in abstract and remove the <s> and </s> tokens at the beginning and end of the sentence
def abstract_combiner(abstract):
    combined = ""
    for sentence in abstract:
        combined += (sentence[4:-4]) + " "
    return combined

In [12]:
# papers[0].keys()

In [13]:
# abstract_combiner(papers[0]["abstract_text"])
# extractive(papers[0]['sections'], num_sentences=4)

In [14]:
# CSV dataset
# import csv
# # Define the header and data separately
# header = ["instruction", "input", "output"]
# 
# # Path to save the CSV file
# file_path = 'sample-data.csv'
# instruction = "Summarize"
# # Open the file in write mode
# with open(file_path, 'w', newline='') as file:
#     writer = csv.writer(file)
#     # Write the header
#     writer.writerow(header)
#     # Write the data rows
#     for row in papers[:10]: # Do for sample 10 papers
#         input = extractive(row['sections'], num_sentences=4)
#         output = abstract_combiner(row["abstract_text"])
#         writer.writerow([row['article_id'], instruction, input, output])

In [15]:
# # JSON for small data (Load entire dataset in-memory)
# from tqdm import tqdm
# json_data = []
# 
# instruction = "Produce a clear and concise abstract summarizing the key points from the input, emphasizing the objectives, methodology, results, and conclusions without detailed technical jargon or formulas. Ignore @xmath terms."
# for row in tqdm(papers[:1000]): # Do for sample 10 papers
#     input = extractive(row['sections'], num_sentences=5)
#     output = abstract_combiner(row["abstract_text"])
#     article_id = row["article_id"]
#     section_names = row["section_names"]
#     json_data.append({
#         "article_id": article_id, 
#         "instruction": instruction, 
#         "input": input, 
#         "output": output,
#         "section_names": section_names})
#     
# # Path to save the JSON file
# file_path = 'sample_data.json'
# with open(file_path, 'w') as file:
#     # json.dump(json_data, file, separators=(',', ':'))
#     json.dump(json_data, file, indent=4)

In [16]:
# Convenience method
instruction = "Produce a clear and concise abstract summarizing the key points from the input, emphasizing the objectives, methodology, results, and conclusions without detailed technical jargon or formulas. Ignore @xmath terms."
def preprocess_function(paper):
    input = extractive(paper['sections'], num_sentences=5)
    output = abstract_combiner(paper["abstract_text"])
    return {
    "article_id": paper["article_id"], 
    "instruction": instruction, 
    "input": input, 
    "output": output,
    "section_names": paper["section_names"]}

In [17]:
# # Modularized method for small JSON data (Still In-memory)
# file_path = 'test_data.json'
# with open(file_path, 'w') as file:
#     file.write('[')
#     for i, item in enumerate(tqdm(papers[:10], desc="Processing items")):
#         processed_item = preprocess_function(item)
#         json_string = json.dumps(processed_item, separators=(',', ':'))
#         if i != 0:
#             file.write(',')
#         file.write(json_string)  
#     file.write(']')

In [ ]:
# Sequential Disk-to-Disk implementation 
# Note: Multi-processing seems to be slowing things down.
import json
from tqdm import tqdm

in_file_path = 'train.json'
out_file_path = 'train_data.json'

with open(in_file_path, 'r') as infile:
    with open(out_file_path, 'w') as outfile:
        outfile.write('[')
        for i, line in enumerate(tqdm(infile, desc="Processing items")):
            try:
                # Load, process, and serialize line to JSON string
                paper_data = json.loads(line) 
                processed_item = preprocess_function(paper_data)
                json_string = json.dumps(processed_item, separators=(',', ':'))
                if i != 0:
                    outfile.write(',')
                outfile.write(json_string)  
            except json.JSONDecodeError as e:
                # Output an error message if a line is not valid JSON
                print(f"Error parsing JSON for line: {line}")
                print(str(e))
            except ValueError as e:
                print(f"Error in extractive summarization for: {paper_data['article_id']}")
                print(e)
        outfile.write(']')

Processing items: 1036it [01:49, 14.07it/s]

Error in extractive summarization for: 1608.03781
empty vocabulary; perhaps the documents only contain stop words


Processing items: 1237it [02:12,  8.15it/s]